# Agenda:

* Data read and feature transformations
* Keras Model
* Keras Training loop

## Import Necessary Libraries

In [1]:
!pip install keras

In [2]:
import keras
print(keras.__version__)

2.13.1


In [3]:
!pip install tensorflow

  Obtaining dependency information for typing-extensions<4.6.0,>=3.6.6 from https://files.pythonhosted.org/packages/31/25/5abcd82372d3d4a3932e1fa8c3dbf9efac10cc7c0d16e78467460571b404/typing_extensions-4.5.0-py3-none-any.whl.metadata
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.34 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
torch 2.4.1 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.23.3 requires typing-

In [4]:
import tensorflow 
print(tensorflow.__version__)

2.13.0


In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

### Part 1: Import the Housing data and do feature transformations

In [6]:
df = pd.read_csv("house_price_full.csv")
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


#### Feature transformations

In [7]:
x = df.copy()

# Remove target from x and store in Y
Y = x.pop('price')

# Perform a scaler transform of the input data
scaler = StandardScaler()
x = scaler.fit_transform(x)

# Perform log transformations of target variable
Y = np.log(Y)

In [8]:
df_scaled = pd.DataFrame(x)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [9]:
Y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

### Part 2: Create Model Using `keras` without `Dropout`

In [10]:
from tensorflow import keras

In [13]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            10, activation="relu", input_shape=(x.shape[-1],)
        ),
#         keras.layers.Dropout(0.2),
        keras.layers.Dense(
            10, activation="relu"
        ),
        keras.layers.Dense(
            5, activation="relu"
        ),

        keras.layers.Dense(1, activation="linear")
    ]
)

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                30        
                                                                 
 dense_5 (Dense)             (None, 10)                110       
                                                                 
 dense_6 (Dense)             (None, 5)                 55        
                                                                 
 dense_7 (Dense)             (None, 1)                 6         
                                                                 
Total params: 201 (804.00 Byte)
Trainable params: 201 (804.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error",
    metrics=['mse']
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                30        
                                                                 
 dense_5 (Dense)             (None, 10)                110       
                                                                 
 dense_6 (Dense)             (None, 5)                 55        
                                                                 
 dense_7 (Dense)             (None, 1)                 6         
                                                                 
Total params: 201 (804.00 Byte)
Trainable params: 201 (804.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Part 3: Model Training

In [19]:
X_train, X_val, y_train, y_val = train_test_split(df_scaled, Y, test_size=0.1, random_state=42)

In [20]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=1)

Epoch 1/10
15/15 [==============================] - 0s 7ms/step - loss: 59.0722 - mse: 59.0722 - val_loss: 3.6695 - val_mse: 3.6695
Epoch 2/10
15/15 [==============================] - 0s 3ms/step - loss: 2.3656 - mse: 2.3656 - val_loss: 0.4280 - val_mse: 0.4280
Epoch 3/10
15/15 [==============================] - 0s 2ms/step - loss: 0.4129 - mse: 0.4129 - val_loss: 1.4765 - val_mse: 1.4765
Epoch 4/10
15/15 [==============================] - 0s 3ms/step - loss: 0.6880 - mse: 0.6880 - val_loss: 0.1283 - val_mse: 0.1283
Epoch 5/10
15/15 [==============================] - 0s 1ms/step - loss: 0.2615 - mse: 0.2615 - val_loss: 0.6992 - val_mse: 0.6992
Epoch 6/10
15/15 [==============================] - 0s 1ms/step - loss: 0.4848 - mse: 0.4848 - val_loss: 0.4397 - val_mse: 0.4397
Epoch 7/10
15/15 [==============================] - 0s 1ms/step - loss: 0.2174 - mse: 0.2174 - val_loss: 0.1214 - val_mse: 0.1214
Epoch 8/10
15/15 [==============================] - 0s 2ms/step - loss: 0.2135 - mse: 0.

In [ ]:
# loss: 0.2803 - mse: 0.2803 - val_loss: 0.1606 - val_mse: 0.1606 -> No dropout layer

### Part 4: Create Model Using `keras` with `Dropout`

In [21]:
model1 = keras.Sequential(
    [
        keras.layers.Dense(
            10, activation="relu", input_shape=(x.shape[-1],)
        ),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(
            10, activation="relu"
        ),
        keras.layers.Dense(
            5, activation="relu"
        ),

        keras.layers.Dense(1, activation="linear")
    ]
)

In [22]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 10)                30        
                                                                 
 dropout_1 (Dropout)         (None, 10)                0         
                                                                 
 dense_9 (Dense)             (None, 10)                110       
                                                                 
 dense_10 (Dense)            (None, 5)                 55        
                                                                 
 dense_11 (Dense)            (None, 1)                 6         
                                                                 
Total params: 201 (804.00 Byte)
Trainable params: 201 (804.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model1.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error",
    metrics=['mse']
)
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 10)                30        
                                                                 
 dropout_1 (Dropout)         (None, 10)                0         
                                                                 
 dense_9 (Dense)             (None, 10)                110       
                                                                 
 dense_10 (Dense)            (None, 5)                 55        
                                                                 
 dense_11 (Dense)            (None, 1)                 6         
                                                                 
Total params: 201 (804.00 Byte)
Trainable params: 201 (804.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Part 5: Model Training

In [25]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=1)

Epoch 1/10
15/15 [==============================] - 0s 4ms/step - loss: 0.2076 - mse: 0.2076 - val_loss: 0.1154 - val_mse: 0.1154
Epoch 2/10
15/15 [==============================] - 0s 999us/step - loss: 0.1697 - mse: 0.1697 - val_loss: 0.1943 - val_mse: 0.1943
Epoch 3/10
15/15 [==============================] - 0s 1ms/step - loss: 0.1693 - mse: 0.1693 - val_loss: 0.1193 - val_mse: 0.1193
Epoch 4/10
15/15 [==============================] - 0s 1ms/step - loss: 0.1657 - mse: 0.1657 - val_loss: 0.8103 - val_mse: 0.8103
Epoch 5/10
15/15 [==============================] - 0s 1ms/step - loss: 0.3575 - mse: 0.3575 - val_loss: 1.4183 - val_mse: 1.4183
Epoch 6/10
15/15 [==============================] - 0s 1ms/step - loss: 0.4270 - mse: 0.4270 - val_loss: 0.1281 - val_mse: 0.1281
Epoch 7/10
15/15 [==============================] - 0s 1ms/step - loss: 0.1592 - mse: 0.1592 - val_loss: 0.4230 - val_mse: 0.4230
Epoch 8/10
15/15 [==============================] - 0s 1ms/step - loss: 0.2363 - mse: 0.

In [27]:
# loss: 0.2803 - mse: 0.2803 - val_loss: 0.1606 - val_mse: 0.1606 -> No dropout layer
# loss: 0.2403 - mse: 0.2403 - val_loss: 0.1413 - val_mse: 0.1413 -> With dropout layer